In [2]:
import numpy as np

#LU decomposition subroutine
def getL(LU):
    dim = LU.shape[0]
    L = LU.copy()
    for i in range(dim):
            L[i, i] = 1
            L[i, i+1:] = 0
    return L

def getU(LU):
    dim = LU.shape[0]
    U = LU.copy()
    for i in range(1, dim):
        U[i, :i] = 0
    return U

def getLU(a, pres):
    dim = a.shape[0]
    a = a.copy()
    detSight = 1
    P = np.matrix(np.zeros(shape=(dim, dim)))
    Q = np.matrix(np.zeros(shape=(dim, dim)))
    np.fill_diagonal(P, 1)
    np.fill_diagonal(Q, 1)
    for i in range(dim - 1):
        (choseRow, choseCol) = np.unravel_index(np.argmax(np.abs(a[i:, i:])), a[i:, i:].shape)
        choseRow += i
        choseCol += i
        if choseRow != i:
            detSight *= -1
            a[[choseRow, i]] = a[[i, choseRow]]
            P[:, [choseRow, i]] = P[:, [i, choseRow]]
        if choseCol != i:
            detSight *= -1
            a[:, [choseCol, i]] = a[:, [i, choseCol]]
            Q[[choseCol, i]] = Q[[i, choseCol]]
        if np.abs(a[i, i]) < pres:
            return P, getL(a), getU(a), Q, detSight
        for j in range(i + 1, dim):
            coef = a[j, i] / a[i, i]
            a[j, i:] = a[j, i:] - a[i, i:]*coef
            a[j, i] = coef
    return P, getL(a), getU(a), Q, detSight

#solving
def directSub(L, B, P):
    dim = P.shape[0]
    B = P.transpose()*B
    Y = np.matrix(np.zeros(shape=(dim, 1)))
    for i in range(dim):
        summ = 0
        for j in range(i):
            summ += L[i, j]*Y[j, 0]
        Y[i, 0] = B[i, 0] - summ
    return Y

def inverceSub(U, Y, P, Q):
    dim = P.shape[0]
    X = np.matrix(np.zeros(shape=(dim, 1)))
    for i in range(dim):
        summ = 0
        for j in range(i):
            summ += U[dim - 1 - i, dim - 1 - j] * X[dim - 1 - j, 0]
        X[dim - 1 - i, 0] = (Y[dim - 1 - i, 0] - summ) / U[dim - 1 - i, dim - 1 - i]
    return (X.transpose()*Q).transpose()

def solve(a, b, pres):
    P, L, U, Q, Det = getLU(a, pres)
    Y = directSub(L, b, P)
    X = inverceSub(U, Y, P, Q)
    return X

#INVIT
def inverce_power_method(matrix_a, appr_sigma, max_iter, tolerance):
    dim = matrix_a.shape[0]
    y = [None]*2
    z = [None]*2
    sigma = [None]*2
    mu = [None]*2

    #1
    y[0] = np.matrix(np.random.rand(dim, 1))
    sigma[0] = appr_sigma
    z[0] = y[0] / np.linalg.norm(y[0])
    mu[0] = np.matrix(np.random.rand(dim, 1))

    for i in range(1, max_iter):
        #2
        y[1] = solve((matrix_a - np.eye(dim)*sigma[0]), z[0], tolerance)
        z[1] = y[1] / np.linalg.norm(y[1])
        #3
        mu[1] = np.divide(z[0], y[1])
        sigma[1] = sigma[0] + np.mean(mu[1])
        #4
        if np.linalg.norm(mu[1]) < tolerance:
            print(f"INVIT ended on {i} iteration", "\n")
            break
        else:
            y[0] = y[1]
            z[0] = z[1]
            sigma[0] = sigma[1]
            mu[0] = mu[1]

    return sigma[1], z[1]

In [5]:
n = np.random.randint(3, 5)
lambdas = np.random.rand(n)*n*100
Alpha = np.diag(lambdas)
C = np.matrix(np.random.randn(n, n))*n
A = np.linalg.inv(C)*Alpha*C

appr_lambdas = lambdas + np.random.rand(n)*0.1
print(lambdas)
print(appr_lambdas, "\n")

found_lambdas = []
for elem in appr_lambdas:
    lambda1, x1 = inverce_power_method(A, elem, 10000, 0.1**12)
    found_lambdas.append(lambda1)

print(found_lambdas - lambdas)

[163.28249136 124.77575429 221.17531415]
[163.30201037 124.78779193 221.18564064] 

INVIT ended on 5 iteration 

INVIT ended on 5 iteration 

INVIT ended on 4 iteration 

[ 0.00000000e+00  2.84217094e-14 -2.84217094e-14]
